## Network Analysis

This notebook aims to visualize the relationship that exists within the anime studios, the anime staff, and tags for each title.

In [2]:
# !pip install nx_altair

In [7]:
import numpy as np
import pandas as pd
import altair as alt
import networkx as nx
import nx_altair as nxa
import matplotlib.pyplot as plt
from networkx.algorithms.community import greedy_modularity_communities
import itertools

In [8]:
# load
df_titles = pd.read_csv("../assets/titles_2000p.csv")
df_mlist = pd.read_csv("../assets/media_list_all_users.csv")
display(df_titles.head())
display(df_mlist.head())

,title_id,title_english,title_romaji,type,duration,start_year,chapters,volume,publishing_status,country,...,score_100,count_CURRENT,count_PLANNING,count_COMPLETED,count_DROPPED,count_PAUSED,ranking_RATED,ranking_POPULAR,synopsis,cover_image_url
0,30002,Berserk,Berserk,MANGA,NaN,1989.0,NaN,NaN,RELEASING,JP,...,28254.0,55869,36200,13918,1704,10641,1.0,5.0,"His name is Guts, the Black Swordsman, a feare...",{'large': 'https://s4.anilist.co/file/anilistc...
1,31706,NaN,JoJo no Kimyou na Bouken: Steel Ball Run,MANGA,NaN,2004.0,95.0,24.0,FINISHED,JP,...,14637.0,6333,14663,30660,355,1487,1.0,1.0,"Originally presented as an unrelated story, th...",{'large': 'https://s4.anilist.co/file/anilistc...
2,114129,Gintama: THE VERY FINAL,Gintama: THE FINAL,ANIME,104.0,2021.0,NaN,NaN,FINISHED,JP,...,5273.0,702,12440,10616,131,146,1.0,7.0,<i>Gintama: THE FINAL</i> is the 3rd and final...,{'large': 'https://s4.anilist.co/file/anilistc...
3,30013,One Piece,ONE PIECE,MANGA,NaN,1997.0,NaN,NaN,RELEASING,JP,...,29853.0,80409,15977,4344,3211,9142,1.0,1.0,"As a child, Monkey D. Luffy was inspired to be...",{'large': 'https://s4.anilist.co/file/anilistc...
4,30656,Vagabond,Vagabond,MANGA,NaN,1999.0,NaN,NaN,RELEASING,JP,...,10628.0,18781,33121,10862,755,6190,1.0,1.0,"At seventeen years of age, Miyamoto Musashi--s...",{'large': 'https://s4.anilist.co/file/anilistc...


,list_id,user_id,title_id,status,repeat
0,5318,1,1,COMPLETED,0
1,5321,1,30,COMPLETED,0
2,5325,1,79,COMPLETED,0
3,5333,1,226,COMPLETED,0
4,5343,1,339,COMPLETED,0


In [9]:
# load
df_studios = pd.read_csv("../assets/studios_200p.csv")
df_staff = pd.read_csv("../assets/staff_200p.csv")
df_tags = pd.read_csv("../assets/tags_200p.csv")

In [10]:
display(df_studios.head(20))

,title_id,title_english,title_romaji,studio_id,studio_name
0,114129,Gintama: THE VERY FINAL,Gintama: THE FINAL,6116,Bandai Namco Pictures
1,114129,Gintama: THE VERY FINAL,Gintama: THE FINAL,6570,Shueisha
2,114129,Gintama: THE VERY FINAL,Gintama: THE FINAL,17,Aniplex
3,114129,Gintama: THE VERY FINAL,Gintama: THE FINAL,415,Warner Bros. Japan
4,114129,Gintama: THE VERY FINAL,Gintama: THE FINAL,53,Dentsu
5,114129,Gintama: THE VERY FINAL,Gintama: THE FINAL,16,TV Tokyo
6,124194,Fruits Basket The Final Season,Fruits Basket: The Final,73,TMS Entertainment
7,124194,Fruits Basket The Final Season,Fruits Basket: The Final,6663,TMS Music
8,124194,Fruits Basket The Final Season,Fruits Basket: The Final,102,Funimation
9,124194,Fruits Basket The Final Season,Fruits Basket: The Final,6429,Avex Pictures


In [11]:
print("total rows: ", len(df_studios), len(df_staff))
print("unique title counts: ", len(df_studios["title_romaji"].unique()), len(df_staff["title_romaji"].unique()))

# There are duplicate rows for each title: for example, "Shinji Takeuchi" appears twice for title "Gintama: THE FINAL"
df_staff_clean = df_staff.drop_duplicates(["title_romaji", "staff_name"])
df_studios_clean = df_studios.drop_duplicates(["title_romaji", "studio_name"])
print("total rows for studios and staff after cleaning: ", len(df_studios_clean), len(df_staff_clean))

total rows:  12219 80967
unique title counts:  3780 8045
total rows for studios and staff after cleaning:  11259 63523


In [12]:
def create_edge_df(df, base_col, pattern_col, r):
    '''
    for every "base_col", it creates a list of possible combinations of "pattern_col" and 
    converts it to df. "r" represents the "r" from nCr
    '''
    res = []
    for t in df[base_col].unique():
        df_filtered = df[df[base_col]==t]
        pattern_set = set(df_filtered[pattern_col])
        pattern_list = list(itertools.combinations(pattern_set, r))
        pattern_list = [sorted([a,b]) for a,b in pattern_list]
        res.append(pattern_list)
    df_res = pd.DataFrame(list(itertools.chain.from_iterable(res)))
    df_res.columns = ["target_0", "target_1"]
    df_weight = df_res.groupby(["target_0", "target_1"]).size().reset_index()
    df_weight.columns = ["target_0", "target_1", "weight"]
    return df_weight

def create_graph(df):
    g = nx.from_pandas_edgelist(df, "target_0", "target_1", create_using=nx.Graph(), edge_attr="weight")
    for n in g.nodes():
        g.nodes[n]["name"] = n
        g.nodes[n]["edge_count"] = len(g.edges(n))
        sorted_weights_list = sorted(g.edges(n, data=True), key=lambda x: x[2]["weight"],reverse=True)
        g.nodes[n]["top_3_associations"] = sorted_weights_list[:3]
    return g

# TODO: fix this function
def search_associates(df, given_name):
    '''
    '''
    df_filtered = df[(df["target_0"]==given_name) | (df["target_1"]==given_name)]
    g = nx.from_pandas_edgelist(df_filtered, "target_0", "target_1", create_using=nx.Graph(), edge_attr="weight")
    for n in g.nodes():
        g.nodes[n]["name"] = n
        g.nodes[n]["edge_count"] = len(g.edges(n))
    return g

def interactive_network_viz(g, tool_tip_list=None, node_color="#FFAA00", edge_color="#1f77b4"):
    pos = nx.kamada_kawai_layout(g)
    e = nxa.draw_networkx_edges(g, pos=pos)
    n = nxa.draw_networkx_nodes(g, pos=pos)
    n = n.mark_circle(color=node_color, opacity=0.8).encode(
        size=alt.Size("edge_count:Q", scale=alt.Scale(range=[50, 1000])),
        tooltip=tool_tip_list
    )
    e = e.mark_line(color=edge_color).encode(
        strokeWidth=alt.Size("weight:Q", scale=alt.Scale(range=[0.1, 5]), legend=None)
    )
    viz = (e+n).properties(width=800, height=800)
    return viz

In [13]:
# # quick vizualization of how one studio is related to another
# print("null count for title_romaji: ", len(df_studios_clean[df_studios_clean["title_romaji"].isnull()])) # check there's no null
# df_weight = create_edge_df(df_studios_clean, "title_romaji", "studio_name", 2)
# studio_g = search_associates(df_weight, "Aniplex")
# interactive_network_viz(studio_g, ["name", "edge_count"])

In [14]:
# limit the titles to "major" titles -> titles with more than 5 studios?
df_weight = create_edge_df(df_studios_clean, "title_romaji", "studio_name", 2)
df_weight_top = df_weight[df_weight["weight"]>5]
g = create_graph(df_weight_top)
interactive_network_viz(g, ["name", "edge_count", "top_3_associations"])

alt.LayerChart(...)

In [15]:
# try similar visualizations with staff data
df_staff_weight = create_edge_df(df_staff_clean, "title_romaji", "staff_name", 2)
df_staff_weight_top = df_staff_weight[df_staff_weight["weight"]>7]
g = create_graph(df_staff_weight_top)
interactive_network_viz(g, ["name", "edge_count", "top_3_associations"], "#2ca02c", "#d62728")

alt.LayerChart(...)

In [16]:
# see if I can do similar viz for other data -> tags
display(df_tags.head())
print("unique tag_name count: ", len(df_tags["tag_name"].unique()))
print("unique tag_category count: ", len(df_tags["tag_category"].unique()))

# Delete duplicate rows
print("total rows before cleaning: ", len(df_tags))
df_tag_name_clean = df_tags.drop_duplicates(["title_romaji", "tag_name"])
print("total rows after cleaning: ", len(df_tag_name_clean))

,tag_id,tag_name,tag_category,tag_rank,title_id,title_english,title_romaji
0,85,Tragedy,Theme-Drama,96,30002,Berserk,Berserk
1,50,Seinen,Demographic,95,30002,Berserk,Berserk
2,252,Revenge,Theme-Drama,93,30002,Berserk,Berserk
3,82,Male Protagonist,Cast-Main Cast,93,30002,Berserk,Berserk
4,15,Demons,Cast-Traits,92,30002,Berserk,Berserk


unique tag_name count:  329
unique tag_category count:  26
total rows before cleaning:  79721
total rows after cleaning:  68238


In [17]:
# Try with tag_name first: Limit the visualization down to popularly associated tags only
df_tags_weight = create_edge_df(df_tag_name_clean, "title_romaji", "tag_name", 2)

# Filter the data down to top 5 tag_name combinations so that we can visualize
df_tags_weight = df_tags_weight.sort_values("weight", ascending=False).groupby("target_0").head(3)
df_tags_weight_top = df_tags_weight[df_tags_weight["weight"]>30]
g = create_graph(df_tags_weight_top)
interactive_network_viz(g, ["name", "edge_count", "top_3_associations"], "#bcbd22", "#9467bd")

alt.LayerChart(...)

## Failed attempt to visualizing title network

In [18]:
# load
df_mlist = pd.read_csv("../assets/media_list_all_users.csv")
df_titles = pd.read_csv("../assets/titles_2000p.csv")
df_mlist_titles = pd.merge(df_mlist, df_titles[["title_id","title_romaji"]], how="inner", on="title_id")
display(df_mlist_titles.head())

,list_id,user_id,title_id,status,repeat,title_romaji
0,5318,1,1,COMPLETED,0,Cowboy Bebop
1,635748,2,1,PLANNING,0,Cowboy Bebop
2,6939,9,1,COMPLETED,0,Cowboy Bebop
3,893937,10,1,COMPLETED,0,Cowboy Bebop
4,398040,11,1,COMPLETED,0,Cowboy Bebop


In [19]:
# limit it to df where its status is "COMPLETED"
print(len(df_mlist_titles))
df_mlist_titles = df_mlist_titles[df_mlist_titles["status"]=="COMPLETED"]
print(len(df_mlist_titles))

# limit the data to only those users with high engagement (more than 50 title registrations)
users_to_keep = df_mlist_titles["user_id"].value_counts().loc[lambda x: x > 50].reset_index()["index"]

# limit the data to only popular titles
print(len(df_mlist_titles["title_id"].unique()))
titles_to_keep = df_mlist_titles["title_id"].value_counts().loc[lambda x: x > 500].reset_index()["index"]

# filter
df_mlist_titles_limit = df_mlist_titles[(df_mlist_titles["user_id"].isin(users_to_keep)) & (df_mlist_titles["title_id"].isin(titles_to_keep))]
print(len(df_mlist_titles_limit))

# get random "n" users
print(len(df_mlist_titles_limit["user_id"].unique()))
idx = np.random.randint(len(df_mlist_titles_limit["user_id"].unique()), size=300) # 300 users
random_users = df_mlist_titles_limit["user_id"].unique()[idx]

df_mlist_titles_limit = df_mlist_titles_limit[df_mlist_titles_limit["user_id"].isin(random_users)]
print(len(df_mlist_titles_limit))
display(df_mlist_titles_limit)

2541925
1536174
32911
711179
4373
47172


,list_id,user_id,title_id,status,repeat,title_romaji
13,28195454,36,1,COMPLETED,0,Cowboy Bebop
63,22799835,172,1,COMPLETED,0,Cowboy Bebop
99,37501262,259,1,COMPLETED,0,Cowboy Bebop
105,3029546,273,1,COMPLETED,0,Cowboy Bebop
148,73353,396,1,COMPLETED,0,Cowboy Bebop
...,...,...,...,...,...,...
2093143,74597486,8208,20629,COMPLETED,0,Terra Formars
2093172,189283447,8539,20629,COMPLETED,0,Terra Formars
2093173,189283447,8539,20629,COMPLETED,0,Terra Formars
2093180,6390903,8595,20629,COMPLETED,0,Terra Formars


In [20]:
df_weight = create_edge_df(df_mlist_titles_limit, "user_id", "title_romaji", 2)
g = create_graph(df_weight)

In [ ]:
# interactive_network_viz(g, ["name", "edge_count"])

pos = nx.random_layout(g)
plt.figure(3,figsize=(12,12)) 
nx.draw_networkx(g, pos=pos)
plt.show()
# e = nxa.draw_networkx_edges(g, pos=pos)
# n = nxa.draw_networkx_nodes(g, pos=pos)
# n = n.mark_circle(color=node_color, opacity=0.8).encode(
#     size=alt.Size("edge_count:Q", scale=alt.Scale(range=[50, 1000])),
#     tooltip=tool_tip_list
# )
# e = e.mark_line(color=edge_color).encode(
#     strokeWidth=alt.Size("weight:Q", scale=alt.Scale(range=[0.1, 5]), legend=None)
# )
# viz = (e+n).properties(width=800, height=800)

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 12444 (\N{KATAKANA-HIRAGANA SEMI-VOICED SOUND MARK}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 12301 (\N{RIGHT CORNER BRACKET}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 65290 (\N{FULLWIDTH ASTERISK}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
